In [1]:
import torch
import torch.nn as nn
import scipy.optimize as optimize
import data.datasets as dataSource
import torch.utils.data as tdata
import numpy as np
import cvxpy as cp

Designer:
- mins and max in dataset, split dataset in two 

Here: 
- get Mask with "important_columns"
- MLP with batchnorm and initial batchnorm (should replace need for scaler)
- train + val loop for new network
- constraints from min, max
- constraints from designer.py in magnetdesigner
- project_to_polytope with optimize.minimize (https://stackoverflow.com/questions/17009774/quadratic-program-qp-solver-that-only-depends-on-numpy-scipy)
- input optim loop without optimizer:
    - loss_fun and tuple of interest as in other pynb
    - create x with requires_grad
    - for step in steps
        - x.grad.zero_()
        - for param in network param.grad.zero_()
        - forward, loss, backward
        - x = x - lr * x.grad
        - x = project_to_poly()
        - x.requires_grad_(True)    

In [2]:
dataset_train, dataset_val = dataSource.Dipole_H_train, dataSource.Dipole_H_val
important_cols = ["aper_x", "aper_y", "aper_x_poleoverhang", "aper_x_tapering", "yoke_x", "yoke_y", "w", "w_leg", "totalCurrent"]
input_mask = [True if col in important_cols else False for col in dataset_train.input_columns]
train_loader, val_loader = tdata.DataLoader(dataset_train,64), tdata.DataLoader(dataset_val,64)

target_index_dict = {col:i for i,col in enumerate(dataset_train.target_columns)}


In [3]:
np.min(dataset_train.dataframe["coil_height"]/dataset_train.dataframe["coil_width"])

0.49999859875289016

In [4]:
np.min(dataset_train.dataframe["totalCurrent"] / (2*dataset_train.dataframe["coil_height"]**2) /1e6)

0.12499975019728285

In [5]:
class MyRegNet(nn.Module):
    def __init__(self,h_size=16):
        super().__init__()
        self.bn0 = nn.BatchNorm1d(len(important_cols), dtype=torch.double)
        self.fc1 = nn.Linear(len(important_cols),h_size, dtype=torch.double)
        self.nl1 = nn.LeakyReLU()
        self.bn1 = nn.BatchNorm1d(h_size, dtype=torch.double)
        self.do1 = nn.Dropout1d()
        #self.fc2 = nn.Linear(h_size,h_size, dtype=torch.double)
        #self.nl2 = nn.LeakyReLU()
        #self.bn2 = nn.BatchNorm1d(h_size, dtype=torch.double)
        #self.do2 = nn.Dropout1d()
        self.fc3 = nn.Linear(h_size,33, dtype=torch.double)
    def forward(self,x):
        x = self.bn0(x)
        x = self.fc1(x)
        x = self.nl1(x)
        x = self.bn1(x)
        x = self.do1(x)
        #x = self.fc2(x)
        #x = self.nl2(x)
        #x = self.bn2(x)
        #x = self.do2(x)
        x = self.fc3(x)
        return x

network = MyRegNet()

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
n_epochs = 100
lr = 1e-4
optim = torch.optim.Adam(network.parameters(), lr)
loss_fun = nn.MSELoss()

for epoch in range(n_epochs):
    network.train()
    loss_list = []
    for x,y in train_loader:
        selected_x = x[:,input_mask].double()
        pred = network(selected_x)
        loss = loss_fun(pred,y)
        loss_list.append(loss.detach().item())
        optim.zero_grad()
        loss.backward()
        optim.step()
    epoch_train_loss = torch.tensor(loss_list).mean()

    network.eval()
    loss_list = []
    for x,y in val_loader:
        selected_x = x[:,input_mask].double()
        pred = network(selected_x)
        loss = loss_fun(pred,y)
        loss_list.append(loss.detach().item())
    epoch_val_loss = torch.tensor(loss_list).mean()
    print(epoch,epoch_train_loss,epoch_val_loss,end="\r")


KeyboardInterrupt: 

In [8]:
A = []
b = []

def onehot(name, neg):
    out = [0] * len(important_cols)
    out[important_cols.index(name)] = -1 if neg else 1
    return out

for name in important_cols:
    # constraints for minima: x > a --> -x < -a
    A.append(onehot(name,True))
    b.append(-dataset_train.input_mins[name])
    
    # constraints for maxima: x < a
    A.append(onehot(name,False))
    b.append(dataset_train.input_maxs[name])

In [19]:
A_rel = []
b_rel = []

# "aper_x", "aper_y", "aper_x_poleoverhang", "aper_x_tapering", "yoke_x", 
# "yoke_y", "w"     , "w_leg"              , "totalCurrent"]


# aper_x > aper_y --> aper_y - aper_x < 0
A_rel.append([-1, 1, 0, 0, 0, 0, 0, 0, 0])
b_rel.append(0)

# aper_x < 5 * aper_y --> aper_x - 5 * aper_y < 0
A_rel.append([1,-5, 0, 0, 0, 0, 0, 0, 0])
b_rel.append(0)

# w_leg < aper_x --> w_leg - aper_x < 0
A_rel.append([-1, 0, 0, 0, 0, 0, 0, 1, 0])
b_rel.append(0)

## Max-made
# w < 6.6*w_leg --> w - 6.6w_leg < 0
A_rel.append([ 0, 0, 0, 0, 0, 0, 1,-6.6, 0])
b_rel.append(0)

# w_leg < 0.83w --> w_leg - 0.83w < 0
A_rel.append([ 0, 0, 0, 0, 0, 0,-0.83, 1, 0])
b_rel.append(0)

# Fix aspect ratio w/h between 1.9 and 2.1
# make coils squareish (aspect ratio of no more than 2 --> for sides a and be we have a - 2b < 0 and b - 2a < 0)
# coil width = 0.5 yoke_x - 0.5 w - aper_tape - w_leg
# coil height = 0.5 yoke_y - 0.6 aper_y - w_leg

# c_w/c_h > 1.9 --> 1.9*c_h - c_w < 0 --> 
#A_rel.append([ 0, -1.14, 0, 1, -0.5, 0.95, 0.5, -0.9, 0])
#b_rel.append(0)

# c_w/c_h < 2.1 --> c_w - 2.1c_h < 0
#A_rel.append([ 0, 1.26, 0, -1, 0.5, 1.05, -0.5, 1.1, 0])
#b_rel.append(0)

In [20]:
A = A + A_rel
b = b + b_rel

In [21]:
list(zip(A,b))

[([-1, 0, 0, 0, 0, 0, 0, 0, 0], -0.025),
 ([1, 0, 0, 0, 0, 0, 0, 0, 0], 0.3),
 ([0, -1, 0, 0, 0, 0, 0, 0, 0], -0.025),
 ([0, 1, 0, 0, 0, 0, 0, 0, 0], 0.3),
 ([0, 0, -1, 0, 0, 0, 0, 0, 0], -0.0094733),
 ([0, 0, 1, 0, 0, 0, 0, 0, 0], 0.3623584),
 ([0, 0, 0, -1, 0, 0, 0, 0, 0], -0.0009473),
 ([0, 0, 0, 1, 0, 0, 0, 0, 0], 0.0362358),
 ([0, 0, 0, 0, -1, 0, 0, 0, 0], -0.1647062),
 ([0, 0, 0, 0, 1, 0, 0, 0, 0], 4.1469032),
 ([0, 0, 0, 0, 0, -1, 0, 0, 0], -0.0912832),
 ([0, 0, 0, 0, 0, 1, 0, 0, 0], 2.791676),
 ([0, 0, 0, 0, 0, 0, -1, 0, 0], -0.0439465),
 ([0, 0, 0, 0, 0, 0, 1, 0, 0], 1.0247168),
 ([0, 0, 0, 0, 0, 0, 0, -1, 0], -0.0128003),
 ([0, 0, 0, 0, 0, 0, 0, 1, 0], 0.9068187),
 ([0, 0, 0, 0, 0, 0, 0, 0, -1], -397.8873575),
 ([0, 0, 0, 0, 0, 0, 0, 0, 1], 286478.8974095),
 ([-1, 1, 0, 0, 0, 0, 0, 0, 0], 0),
 ([1, -5, 0, 0, 0, 0, 0, 0, 0], 0),
 ([-1, 0, 0, 0, 0, 0, 0, 1, 0], 0),
 ([0, 0, 0, 0, 0, 0, 1, -6.6, 0], 0),
 ([0, 0, 0, 0, 0, 0, -0.83, 1, 0], 0),
 ([0, -1.14, 0, 1, -0.5, 0.95, 0.5, -

In [22]:
#def project_to_poly(x, A_mat=A, b_vec = b):
#    def loss(curr_guess):
#        return np.linalg.norm(curr_guess-x)
#    
#    # We know coil_width = 2*coil_height
#    # Area = 2 coil_width^2 
#
#    # currDens = totalCurr / 2 coil_height^2
#    # 0.12 < currDens < 7.5 
#
#    # sqrt(0.12) < sqrt(totalCurr) / (sqrt(2) coil_height) --> coil_height < sqrt(totalCurr) / (sqrt(2)*sqrt(0.12))
#    # sqrt(7.5) > sqrt(totalCurr) / (sqrt(2) coil_height)) --> coil_height > sqrt(totalCurr) / (sqrt(2)*sqrt(7.5))
#    
#    cons = {'type':'ineq','fun':lambda x: b_vec - np.dot(A_mat,x)}
#    return optimize.minimize(loss, x,constraints=cons, method='SLSQP').x
#
##A_test = [[0,1],[1,0]]
##b_test = [1,1]
##project_to_poly([2,0.1],A_test,b_test)

In [23]:
def project_to_poly2(x, A_mat=A, b_vec = b):
    # Define decision variable
    p = cp.Variable(len(x))
    # Define objective function
    objective = cp.Minimize(cp.sum_squares(p - x))

    A_mat_dynamic = A_mat[:]
    b_vec_dynamic = b_vec[:]


    # Define constraints
    constraints = [np.array(A_mat) @ p <= np.array(b_vec)]

    # Solve the problem
    problem = cp.Problem(objective, constraints)
    problem.solve()


    # Optimal solution
    closest_point = p.value
    return closest_point

A_test = [[0,1],[1,0]]
b_test = [1,1]
project_to_poly2([0.5,1.2],A_test,b_test)

array([0.5, 1. ])

In [24]:
tuple_of_interest = (target_index_dict["B0"], target_index_dict["gfr_x_5e-3"],target_index_dict["gfr_y_5e-3"])

target = torch.zeros(1,33,dtype=torch.double)
target[0,tuple_of_interest[0]] = 0.079589
target[0,tuple_of_interest[1]] = 0.053487
target[0,tuple_of_interest[2]] = 0.01

def create_loss_function(tuple_of_interest):
    mask = torch.zeros(1,33)
    for i in tuple_of_interest:
        mask[0,i] = 1
    mse = torch.nn.MSELoss()
    def instanciated_loss_function(x,y):
        return mse(x*mask,y*mask)
    return instanciated_loss_function

loss_fun = create_loss_function(tuple_of_interest)

In [25]:
network.bn0.running_mean

tensor([1.8497e-01, 1.1192e-01, 1.3988e-01, 1.3988e-02, 1.3683e+00, 8.1847e-01,
        4.6473e-01, 2.4637e-01, 3.7710e+04], dtype=torch.float64)

In [28]:
#magnet_params = torch.randn((1,9),dtype=torch.double,requires_grad=True)
magnet_params = torch.tensor(network.bn0.running_mean.clone().detach(),dtype=torch.double).unsqueeze(0).requires_grad_(True)
lr = 1e-2
network.eval()

n_steps = 1000

for step in range(n_steps):
    if magnet_params.grad is not None:
        magnet_params.grad.zero_()
    for layer in network.parameters():
        if layer.grad is not None:
            layer.grad.zero_()

    pred = network(magnet_params)
    loss = loss_fun(pred,target)
    print(step,loss.detach().numpy(),end="\r")
    loss.backward()
    with torch.no_grad():
        magnet_params = magnet_params - lr * network.bn0.running_mean * magnet_params.grad
        proj_result = project_to_poly2(magnet_params.squeeze().detach().numpy())
        if (proj_result is None):
            magnet_params = torch.tensor(magnet_params).requires_grad_(True)
        else:        
            magnet_params = torch.tensor(proj_result).unsqueeze(0).requires_grad_(True)






/tmp/ipykernel_137566/606218387.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  magnet_params = torch.tensor(network.bn0.running_mean.clone().detach(),dtype=torch.double).unsqueeze(0).requires_grad_(True)


In [29]:
magnet_params

tensor([[1.1782e-01, 2.3563e-02, 1.6525e-01, 3.9011e-02, 1.6332e-01, 2.8847e-02,
         1.9805e-01, 2.9314e-02, 3.7710e+04]], dtype=torch.float64,
       requires_grad=True)